In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# use T4?
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

save_dir = "/content/drive/MyDrive/DSC261 Final Project/opt125m-news-model-base"

# Load model
model = AutoModelForCausalLM.from_pretrained(
    save_dir,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(save_dir)

`torch_dtype` is deprecated! Use `dtype` instead!


In [4]:
import pandas as pd
synthetic = pd.read_csv("/content/drive/MyDrive/DSC261 Final Project/Synthetic Data/synthetic_data_prepared.csv", on_bad_lines='skip', engine='python')

In [29]:
def generate_text(input_text, min_new_tokens=400, max_new_tokens=1100, temperature=0.5):
  input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

  output = model.generate(
      input_ids,
      max_new_tokens=max_new_tokens,
      min_new_tokens=min_new_tokens,
      do_sample=True,
      top_p=0.9,
      temperature=temperature,
      repetition_penalty=1.2,      # important
      no_repeat_ngram_size=3       # VERY important
  )

  return tokenizer.decode(output[0], skip_special_tokens=True)

In [23]:
# find your data section

# change this part
person = 'Sarah'


my_data = synthetic[synthetic['person'] == person]
my_data

,id,article,highlights,article_length,cleaner_article,length_after_clean,person,max_new_tokens,temperature,prefix
0,4bb5b7845ca7a623cc61530dd7eb94a7377b95f1,President Barack Obama is enjoying strong supp...,56 per cent of Americans overall say governmen...,4505,President Barack Obama is enjoying strong supp...,4505,Sarah,1126,0.4,President Barack Obama is enjoying strong supp...
1,5fd789d8d69a1b620991a540fcea1bc0cb578d2c,Staff at the Ministry of Defence have spent £2...,"The £270,000 could have paid for 15 squaddies ...",2736,Staff at the Ministry of Defence have spent £2...,2736,Sarah,684,0.4,Staff at the Ministry of Defence have spent £2...
2,1ed727c7e0414683b5c555d730d89edfd8d85c3e,(CNN) -- A Florida teenager whose videotaped b...,"Tariq Abu Khdeir, 15, was expected to arrive i...",3254,A Florida teenager whose videotaped beating in...,3245,Sarah,811,0.3,A Florida teenager whose videotaped beating in...
3,ebf5fb0892d368ad683a06632704e18dfa1446bf,(CNN) -- Secret Service agents arrested a Wash...,The man is arrested at a Seattle suburb .\nFed...,841,Secret Service agents arrested a Washington ma...,832,Sarah,208,0.6,Secret Service agents arrested a Washington ma...
4,f19b58cfb51a9c893bd7757cd6ae92a66386fd33,"By . James Gordon . For a couple from Iowa, ex...",Jess and Paul from Iowa were heading to hospit...,2741,"For a couple from Iowa, exit 52 on Interstate ...",2721,Sarah,680,0.3,"For a couple from Iowa, exit 52 on Interstate ..."
...,...,...,...,...,...,...,...,...,...,...
14281,aba2ae63ef6b7f27069aa21204a738e0dfb9ce7b,Manchester United midfield Nick Powell has suf...,Manchester United signed Nick Powell for £4mil...,1059,Manchester United midfield Nick Powell has suf...,1059,Sarah,265,0.8,Manchester United midfield Nick Powell has suf...
14282,05412a0ba5af52d5b9bf116804fac7b3a52d6b62,"By . Joshua Gardner . An Oakland, California p...","Oakland, California high school senior Akintun...",2365,"An Oakland, California public school kid who h...",2343,Sarah,586,0.8,"An Oakland, California public school kid who h..."
14283,91207937e6f3290dab41749fd7da1b9efb6cdfbe,(CNN) -- The three Afghan National Army office...,"The men were sighted at Zachary's Pub, a strip...",1807,The three Afghan National Army officers who di...,1798,Sarah,450,0.5,The three Afghan National Army officers who di...
14284,c6d1110c248c9795c36607f35874e56a26166aa4,The woman whose shocked face amid the carnage ...,Fitness instructor and triathlete Nicole Gross...,2793,The woman whose shocked face amid the carnage ...,2793,Sarah,698,0.8,The woman whose shocked face amid the carnage ...


In [41]:
# use the 'prefix' column as your input text
# use max_new_tokens columns for max_new_tokens parameter, this is based on true length of original
# I randomly assigned the temperature between 0.3 - 0.8 but I set it to 0.4, feel free to experiment

In [36]:
# double check your data is clean of headers

In [34]:
# example
generate_text("President Barack Obama is enjoying strong support for his NSA's surveillance program from Democratic partisans. Polls conducted in 2006 and", 600, 684, 0.8)

'President Barack Obama is enjoying strong support for his NSA\'s surveillance program from Democratic partisans. Polls conducted in 2006 and 2007 found that 49% of those surveyed said they were strongly opposed to the use of the agency\'s "no-fly" policy, but only 44% wanted it lifted after an American spy was killed by the nation\'s intelligence agencies. Now President Obama has taken a stance against the controversial program -- which has been used in the past to track foreign fighters who are suspected of terrorist activity and even terrorists convicted on terrorism charges -- calling them an abuse of federal law. The president\'s public comments came as he defended the new collection of data from the Internet. "What we have here today is extraordinary information," Obama told CBS\' "Face the Nation." "And what we have now, at least since I took office, is extraordinary evidence of our ability to collect such enormous amounts of material, including from the internet." He also calle

In [40]:
from tqdm import tqdm
import torch
import os

input_df = my_data.head(10).copy()
# do on your own so to not add too much to shared drive storage
save_dir = "/content/drive/MyDrive/inference_outputs"
os.makedirs(save_dir, exist_ok=True)

# min_new_tokens = max_new_tokens - 100 (min 0)
input_df["min_new_tokens"] = (input_df["max_new_tokens"] - 100).clip(lower=0)

# store outputs
input_df["generated_text"] = ""

def generate_text(prefix, max_nt, min_nt):
    inputs = tokenizer(prefix, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_nt,
            min_new_tokens=min_nt,
            temperature=0.4,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)

# run in batches

batch_size = 50
num_batches = (len(input_df) + batch_size - 1) // batch_size

for b in tqdm(range(num_batches)):
    start = b * batch_size
    end = min((b + 1) * batch_size, len(input_df))

    batch = input_df.iloc[start:end]

    print(f"\n🔹 Processing batch {b+1}/{num_batches} ({start} to {end-1})")

    # generate for each row
    texts = []
    for idx, row in batch.iterrows():
        text = generate_text(
            prefix=row["prefix"],
            max_nt=int(row["max_new_tokens"]),
            min_nt=int(row["min_new_tokens"])
        )
        input_df.at[idx, "generated_text"] = text

    # save partial results
    out_path = f"{save_dir}/generation_batch_{b+1}.csv"
    input_df.iloc[:end][["id", "generated_text"]].to_csv(out_path, index=False)
    print(f"💾 Saved progress to {out_path}")

# final full save
final_path = f"{save_dir}/full_generation_output.csv"
input_df[["id", "generated_text"]].to_csv(final_path, index=False)
print(f"\n🎉 DONE! Full output saved to:\n{final_path}")


  0%|          | 0/1 [00:00<?, ?it/s]


🔹 Processing batch 1/1 (0 to 9)


100%|██████████| 1/1 [01:07<00:00, 67.52s/it]

💾 Saved progress to /content/drive/MyDrive/inference_outputs/generation_batch_1.csv

🎉 DONE! Full output saved to:
/content/drive/MyDrive/inference_outputs/full_generation_output.csv
